In [1]:
#!conda activate py38_estnltk1.7
#hpc serveris peaks sobima estnltk_collocations_py38
#!conda install --channel conda-forge pygraphviz
#!conda install -c conda-forge ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

# Verb - compound - obl - obl_case kollokatsioonide kogumine koondkorpusest


# v20
Kollokatsiooniandmete salvestamisel tabelisse jäeti välja kollokatsioonid, milles:

* verb oli umbisikuline (<code>feats</code> sisaldas parameetrit <code>imps</code>);
* verbi aeg polnud ükski järgnevatest: <code>past</code>, <code>impf</code>, <code>pres</code> (<code>feats</code> ei sisaldanud parameetreid <code>past</code> ega <code>impf</code> ega  <code>pres</code>).

### Tabelid

### TABEL1 
**verb\_compound\_obl**

Paksu kirjaga on märgitud väljad, mis moodustavad unikaalse indeksi.

| väli | tüüp  |  kirjeldus | näide | märkus |
| --- | --- | --- | --- | --- |
| id | int | kollokatsiooni<br/>unikaalne ID| *56* | |
| **verb** | text | tegusõna lemma | *kirjutama* | |
| **verb_compound** | text | määrsõna(de) lemma(d) | *alla,kokku* | mitme määrsõna korral on lemmade eraldajaks koma ja lemmad on alfabeetilises järjestuses |
| **obl_root** | text| täiendi juure lemma nn "puhastatud kuju"| *kurjajuur* | |
| **obl\_root\_compound** | text| täiendi juure lemma liitsõnamärkidega*| *kurja_juur* | |
| **obl_case** | text | täiendi juure kääne | *ad* | vt [Käänded](#käänded) |
| count | int | kollokatsiooni esinemiste arv üle korpuse | *56* |  | 

### TABEL2 

**verb\_compound\_obl\_examples**


| väli | tüüp  |  kirjeldus |	näide | märkus |
| --- | --- | --- | --- | --- |
| **row_id** | int | rea id tabelis TABEL1| | |
| **sentence_id** | int | kollektsiooni id postgres andmebaasis | | |
| **root_id** | int | OBL fraasi juur (sõna nr) | | |
| verb_id | int | verbi id (sõna nr) | | |
| compound_ids | text | määrsõna(de) id(-d) (sõna nr) | | |
| obl_ids | text | täiendi fraasi kõik liikmed | | |



### Käänded
| kääne|  nimetus| 
| --- | --- |
| nom |  nimetav | 
| gen | omastav | 
| part | osastav | 
| adit | lühike sisseütlev | 
| ill | sisseütlev | 
| in |  seesütlev | 
| el |  seestütlev | 
| all |  alaleütlev |
| ad |  alalütlev | 
| abl |  alaltütlev | 
| tr | saav | 
| term |  rajav | 
| es |  olev | 
| abes |  ilma | 
| kom | kaasa | 
| \<käändumatu\> | täiendil puudus kääne | 





In [2]:
%load_ext autoreload
%autoreload 2

from estnltk.storage.postgres import LayerQuery

from datetime import datetime
# functions for creating database and collecting collocations
from collect_functions import *

from data_helpers.db_reader import DbReader

#collection_name = 'koondkorpus_sentences'
#BATCH_SIZE = 50000

collection_name = 'koondkorpus_sentences_test_5000_sg_thread'
BATCH_SIZE = 50000

TYPE = 'verb_compound_obl'
TABLENAME = f'{TYPE}'

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
db_file_name = f"v20_{collection_name}_{TYPE}_collocations_{date_time}.db"
my_sqlite_db = DbMethods(db_file_name=db_file_name, table1_name=TYPE, table2_name=TYPE+'_examples')
my_sqlite_db.prep_coll_db()


my_db_reader = DbReader(pgpass_file='~/.pgpass',\
                          schema='estonian_text_corpora',\
                          role='estonian_text_corpora_read',\
                          temporary=False,\
                          collection_name=collection_name)
my_db_reader.set_layers(['v172_stanza_syntax', 'v172_obl_phrases'])

# vaatame ainult neid lauseid, kus on mõni tegusõna ka
# NB! kas saab olla obl, kus ülemus ei ole tegusõna
additional_filters = [LayerQuery('morph_analysis', partofspeech='V')]
# sisaldab OBL fraasi
additional_filters.append(LayerQuery('v172_stanza_syntax', deprel='obl'))

collocations = {}
for collection_id, text in my_db_reader.get_collections(shuffle=False, queries=additional_filters, progressbar='ascii'): 
    collocations, = extract_something(text, collection_id, collocations )
    if not collection_id == 0 and not collection_id % BATCH_SIZE:
        my_sqlite_db.save_coll_to_db(collocations, collection_id)
        collocations = {}

# saving last batch
my_sqlite_db.save_coll_to_db(collocations, collection_id)
my_sqlite_db.index_fields()

INFO:storage.py:41: connecting to host: 'postgres.keeleressursid.ee', port: '5432', dbname: 'estonian-text-corpora', user: 'zummy'
INFO:storage.py:58: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


collection_id: 41:   0%|                                                                                                                                                                                                                                                                                                                | 23/90854 [00:00<20:00, 75.67doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 10
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 30
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 40


collection_id: 84:   0%|1                                                                                                                                                                                                                                                                                                               | 51/90854 [00:00<17:17, 87.49doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 60
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 70
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 80
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 90


...

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 158520
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 158530
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 158630
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 158770
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 158820


collection_id: 159130: 100%|#######################################################################################################################################################################################################################################################################################################8| 90815/90854 [13:08<00:00, 115.14doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 158890
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 158980
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159060
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159070
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159080
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159130
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159210


collection_id: 159735: 100%|########################################################################################################################################################################################################################################################################################################| 90854/90854 [13:08<00:00, 115.16doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159370
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159450
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159640
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159735


In [3]:
# display(text['morph_analysis'])

In [4]:
# display(text['v172_stanza_syntax'])